In [1]:
cd "/Users/TaihuaLi/Desktop/Projects/Netflix Prize/"

/Users/TaihuaLi/Desktop/Projects/Netflix Prize


In [2]:
import numpy as np
import pandas as pd

In [3]:
ratings = pd.read_csv("Rating Zeros.csv", index_col=0) # row: users  col: movies
items = pd.read_csv("items.csv", index_col=0) # movies information
genre = pd.read_csv("genre.csv") # movie genre information

In [4]:
ratings.head(3)

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
items.head(3)

,Name,Unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantacy,Film.Noir,Horror,Musical,Mystery,Romance,Sci.Fi,Thriller,War,Western
﻿Movie.ID,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
genre.head(3)

,﻿Genre,ID
0,unknown,0
1,Action,1
2,Adventure,2


### K-means Clustering: Users

In [7]:
from sklearn.cluster import KMeans

In [8]:
kmeans = KMeans()

In [9]:
kmeans.fit(ratings)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=8, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [10]:
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [11]:
pd.set_option('display.max_rows', ratings.describe().shape[1])
pd.DataFrame(centroids, index=("Cluster 1", "Cluster 2", "Cluster 3", "Cluster 4", 
                               "Cluster 5", "Cluster 6", "Cluster 7", "Cluster 8"))

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
Cluster 1,2.312500,0.343750,3.750000e-01,2.156250,0.250000,8.437500e-01,2.875000,2.531250,3.125000,1.968750,...,4.336809e-19,-8.673617e-19,4.336809e-19,-4.336809e-19,4.336809e-19,-2.168404e-19,4.336809e-19,-4.336809e-19,4.336809e-19,4.336809e-19
Cluster 2,2.731959,0.015464,3.762887e-01,0.056701,0.025773,1.134021e-01,2.237113,0.103093,1.597938,0.412371,...,2.602085e-18,-1.040834e-17,2.602085e-18,-5.204170e-18,2.602085e-18,-2.602085e-18,2.602085e-18,-5.204170e-18,2.602085e-18,2.602085e-18
Cluster 3,3.634146,2.292683,1.658537e+00,3.341463,1.658537,2.682927e-01,3.926829,3.000000,2.926829,0.804878,...,8.673617e-19,-3.469447e-18,8.673617e-19,-1.734723e-18,8.673617e-19,-8.673617e-19,8.673617e-19,-1.734723e-18,7.317073e-02,7.317073e-02
Cluster 4,1.361582,0.129944,4.519774e-02,0.531073,0.169492,1.016949e-01,1.152542,0.971751,0.740113,0.225989,...,1.694915e-02,-1.040834e-17,-1.301043e-18,-5.204170e-18,-1.301043e-18,-2.602085e-18,-1.301043e-18,-5.204170e-18,-1.301043e-18,-1.301043e-18
Cluster 5,2.619048,1.285714,3.333333e-01,2.404762,0.666667,-2.775558e-17,2.357143,2.785714,1.833333,0.904762,...,4.336809e-19,9.523810e-02,4.336809e-19,-1.734723e-18,4.336809e-19,-8.673617e-19,4.336809e-19,-1.734723e-18,4.336809e-19,4.336809e-19
Cluster 6,0.289855,0.021739,-4.440892e-16,0.043478,0.126812,3.985507e-02,0.268116,0.097826,0.449275,0.094203,...,1.517883e-17,-1.127570e-17,1.517883e-17,-5.637851e-18,1.517883e-17,3.623188e-03,1.086957e-02,7.246377e-03,1.517883e-17,1.517883e-17
Cluster 7,3.400000,2.152941,6.117647e-01,1.988235,0.682353,1.110223e-16,2.447059,1.635294,0.976471,0.341176,...,-4.770490e-18,-8.673617e-18,-4.770490e-18,-4.336809e-18,-4.770490e-18,-2.168404e-18,-4.770490e-18,-4.336809e-18,-4.770490e-18,-4.770490e-18
Cluster 8,2.916667,0.479167,4.791667e-01,1.552083,0.541667,4.166667e-02,2.260417,2.041667,2.291667,0.333333,...,-5.204170e-18,-8.673617e-18,3.125000e-02,2.083333e-02,3.125000e-02,-2.168404e-18,-5.204170e-18,-4.336809e-18,-5.204170e-18,-5.204170e-18


In [12]:
movie_names = items.Name

In [13]:
genre_types = items.columns[1:]

In [14]:
items_array = np.array(items)
movie_genre = np.array(items[genre_types])

In [15]:
items_array

array([['Toy Story (1995)', 0, 0, ..., 0, 0, 0],
       ['GoldenEye (1995)', 0, 1, ..., 1, 0, 0],
       ['Four Rooms (1995)', 0, 0, ..., 1, 0, 0],
       ..., 
       ['Sliding Doors (1998)', 0, 0, ..., 0, 0, 0],
       ['You So Crazy (1994)', 0, 0, ..., 0, 0, 0],
       ['Scream of Stone (Schrei aus Stein) (1991)', 0, 0, ..., 0, 0, 0]], dtype=object)

##### Cluster Definition

In [16]:
def FiveStarMovies(centroids, labels, k, NTerms):
    five_star_movies = []
    cluster_movies = []
    for i in range(k):
        cluster_members = ratings[labels==i]
        print 'cluster',i+1,'size: ', cluster_members.shape[0]
        movie_total_rating = np.array(cluster_members.sum(0)).astype(float)
        movie_rating_count = np.array((cluster_members!=0).sum(0)).astype(float)
        movie_average_rating = np.divide(movie_total_rating, movie_rating_count)
        movie_average_rating[np.isnan(movie_average_rating)] = 0
        rated_movie_list = sorted(zip(movie_names, movie_average_rating), key=lambda x: int(x[1]), reverse=True)
        to_append = []
        cluster_append = []
        for j in rated_movie_list:
            if j[1] == 5:
                to_append.append(j[0])   
            if j[1] != 0:
                cluster_append.append(j[0])
        five_star_movies.append(to_append)
        cluster_movies.append(cluster_append)
    print "\n"
    # most rated movies in each cluster
    k = 0
    for m in five_star_movies:
        prototype = list(np.zeros(len(genre_types)))
        for n in items_array:    
            for i in m:
                if i == n[0]:
                    prototype += n[1:]
        prototype = map(int, prototype)
        movie_genre_to_use = sorted(zip(genre_types, prototype), key=lambda x:int(x[1]), reverse=True)
        print "Cluster", k+1, "top genre: "
        for i in movie_genre_to_use[:NTerms]:
            print i[0]
        print "\n"
        k += 1
    return five_star_movies, cluster_movies

In [17]:
five_star_movies, cluster_movies = FiveStarMovies(centroids, labels, 8, 5)

cluster 1 size:  32
cluster 2 size:  194
cluster 3 size:  41
cluster 4 size:  177
cluster 5 size:  42
cluster 6 size:  276
cluster 7 size:  85
cluster 8 size:  96


Cluster 1 top genre: 
Drama
Comedy
Animation
Musical
Action


Cluster 2 top genre: 
Drama
Romance
Comedy
Thriller
Action


Cluster 3 top genre: 
Drama
Comedy
Documentary
Crime
Thriller


Cluster 4 top genre: 
Drama
Comedy
Romance
Action
Crime


Cluster 5 top genre: 
Drama
Documentary
Romance
Comedy
Crime


Cluster 6 top genre: 
Drama
Comedy
Children
Romance
Adventure


Cluster 7 top genre: 
Drama
Comedy
Romance
Action
Crime


Cluster 8 top genre: 
Drama
Comedy
Romance
Documentary
Sci.Fi


